# Final Project: Honeycomb AKLT as PEPS

Submitted by Yoav Zack, ID 211677398

## `julia` Implementation

We will implement those two wave functions as PEPS in `julia` using `TensorOperations`:

In [ ]:
using LinearAlgebra, TensorOperations, Plots, Printf, BenchmarkTools
theme(:dracula)

First, we define the Honeycomb structure, and some useful functions on it:

In [ ]:
mutable struct LatticeAKLT
    shape::String
    verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}
    bonds::Vector{Tuple{Tuple{Int64, Int64}, Tuple{Int64, Int64}}}
    ψ::Vector{Array{Float64}}

    function LatticeAKLT(shape::String, a::Real, shift::Tuple, σ_boundary::Int64)
        @assert shape ∈ ("hexagon", "tree")
        @assert a > 0 "Bond length `a` must be positive"
        @assert length(shift) == 2 "Lattice shift must be of length 2"
        
        # define verts location for plots
        if shape == "hexagon"
            locs = [a.*(cos(2π/6*n), sin(2π/6*n)) for n in 0:5]
            bonds = [(((ind-1)%6+1,2), (ind%6+1,1)) for (ind,_) in verts]
        elseif shape == "tree"
            locs = [(0.0,0.0)]
            append!(locs, [(sin(2π/3*n), cos(2π/3*n)) for n in 0:2])
            bonds = [((1,ind-1),(ind,1)) for ind in 2:4]
        end
        
        # add shift to vertex locations and define verts array
        verts = [(ind, loc .+ shift) for (ind, loc) in enumerate(locs)]
        
        # define bonds

        # fill in tensors
        ψ = AKLT_PEPS(verts, bonds, σ_boundary)
        
        # create structure
        new(shape, verts, bonds, ψ)
    end
end

In [ ]:
function showlattice(fig::Plots.Plot, lat::LatticeAKLT, dolabel::Bool)
    x = [loc[1] for (ind, loc) in lat.verts]
    y = [loc[2] for (ind, loc) in lat.verts]
    for bond in lat.bonds
        inds = [b[1] for b in bond]
        dx = [lat.verts[i][2][1] for i in inds]
        dy = [lat.verts[i][2][2] for i in inds]
        plot!(fig, dx, dy, label="", color="cyan")
    end
    scatter!(fig, x, y, aspect_ratio=1, label="", color="dark red")

    if dolabel
        for (ind, loc) in lat.verts
            annotate!(fig, loc[1], loc[2], ind)
        end
    end
    return fig
end

The tensors are defined using the following functions:

In [ ]:
function AKLT_PEPS(verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}, bonds::Vector{Tuple{Tuple{Int64, Int64}, Tuple{Int64, Int64}}}, σ_sign::Int64)
    """
    Builds a PEPS tensor array of the AKLT model for a given set of vertices and the bonds between them, assuming a HoneycombLattice structure (i.e. three connections at most)
    """

    @assert σ_sign ∈ [1,2] "The value of boundary virtual spins must be up (1) or down (2)"
    
     #σa Θ tensor per site. Θ tensors have 4 indices: σ,a,b,c. The only real one is σ, the rest are virtual
    ψ = [projectionOp() for i in 1:length(verts)]

    # multiply the tensors by the bond matrices: antisymmetric spin-1/2
    Σ = zeros(2,2); Σ[1,2] = 1/sqrt(2); Σ[2,1] = -1/sqrt(2);
    for bond in bonds
        (ind, conn) = bond[1]
        A = ψ[ind]
        if conn == 1
            @tensor B[σ,ap,b,c] := A[σ,a,b,c] * Σ[a,ap]
        elseif conn == 2
            @tensor B[σ,a,bp,c] := A[σ,a,b,c] * Σ[b,bp]
        elseif conn == 3
            @tensor B[σ,a,b,cp] := A[σ,a,b,c] * Σ[c,cp]
        end
        ψ[ind] = B
    end

    # finally, set the boundary legs of each vertex to specific value, if needed
    for (ind, A) in enumerate(ψ)
        c, _ = countneighbors(verts, verts[ind], bonds)
        if c == 0
            A = reshape(A[:,σ_sign,σ_sign,σ_sign], 4,1,1,1)
        elseif c == 1
            A = reshape(A[:,:,σ_sign,σ_sign], 4,2,1,1)
        elseif c == 2
            A = reshape(A[:,:,:,σ_sign], 4,2,2,1)
        end
        ψ[ind] = A
    end
    return ψ
end

In [ ]:
function projectionOp()
    Θ = zeros(4,2,2,2)
    for a in 1:2
        for b in 1:2
            for c in 1:2
                Θ[1,a,b,c] = (a+b+c == 3)
                Θ[2,a,b,c] = (a+b+c == 4)/sqrt(3)
                Θ[3,a,b,c] = (a+b+c == 5)/sqrt(3)
                Θ[4,a,b,c] = (a+b+c == 6)
            end
        end
    end
    return Θ
end

In [ ]:
function countneighbors(verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}, vert::Tuple{Int64, Tuple{Float64, Float64}}, bonds::Vector{Tuple{Tuple{Int64, Int64}, Tuple{Int64, Int64}}})
    c = 0
    vert_bonds = []
    (ind, loc) = vert
    for bond in bonds
        inds = [b[1] for b in bond]
        if ind ∈ inds
            c += 1
            push!(vert_bonds, bond)
        end
    end
    return c, vert_bonds
end

Let's plot the lattices:

In [ ]:
lat = LatticeAKLT("tree" ,1, (5.0,1.0), 1)
fig = showlattice(scatter(), lat, true)

In the MPS case, we could use SVD to move the orthogonality center to a single site, and then the contraction of all of ther sites will be trivial. Unfortunately, this is not possible with PEPS, since the orthogonality center is not even well defined. So, we will have to perform full contraction each time. For example:

In [ ]:
Sz = diagm([3,1,-1,-3]./2)
I = diagm([1,1,1,1])
lat.verts = [(1,(0.0,0.0)),(2,(1.0,0.0))]
lat.bonds = [((1,1),(2,1))]
lat.ψ = AKLT_PEPS(lat.verts, lat.bonds, 1)
@tensor g[σ1, σ2] := lat.ψ[1][σ1,a,1,1] * lat.ψ[2][σ2,a,1,1]
@tensor sz = g[σ1,σ2] * Sz[σ1,a] * I[σ2,b] * g[a,b]
@tensor n = g[σ1,σ2] * I[σ1,a] * I[σ2,b] * g[a,b]
display(sz/n)

So, let's write a function which takes a lattice and calculates the expectation value for a given site:

In [ ]:
function expect(lat::LatticeAKLT, Op::Matrix, ind::Int64)
    @assert lat.shape == "tree" "hexagon contraction not implemented yet"
    
    if ind == 1
        @tensor g[σ,σ1,σ2,σ3] := lat.ψ[1][σ,a,b,c] * lat.ψ[2][σ1,a,1,1] * lat.ψ[3][σ2,b,1,1] * lat.ψ[4][σ3,c,1,1]
        @tensor Z = g[σ,σ1,σ2,σ3] * g[σ,σ1,σ2,σ3]
        @tensor O = g[σ,σ1,σ2,σ3] * Op[σ,σp] * g[σp,σ1,σ2,σ3]
    else
        Z = 1
        O = 0
    end

    @show O, Z
    return O/Z
end

In [ ]:
a = 1.0
Sz = diagm([3,1,-1,-3]./2)
lat = LatticeAKLT("tree" ,a, (5.0,1.0), 2)
expect(lat, Sz, 1)